In [1]:
library(tidyverse)
library(here)
library(glue)
library(furrr)
suppressPackageStartupMessages(library(rtracklayer))
devtools::load_all(".")

plan(multiprocess, workers = 20)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

here() starts at /mnt/expressions/mp/archaic-ychr


Attaching package: ‘glue’


The following object is masked from ‘package:dplyr’:

    collapse


Loading required package: future

Loading ychr



# Full 6.9 Mb capture

In [2]:
samples_full <- list.files(here("data/vcf"), "full_.*.vcf.gz$") %>%
    str_subset("mez2_dp|modern|merged|deam", negate = T) %>%
    str_replace("full_(.*).vcf.gz", "\\1")

In [3]:
samples_full

[1] "a00_1"             "a00_2"             "a00"              
 [4] "chimp"             "den4_snpad"        "den4"             
 [7] "den8_snpad"        "den8"              "mez2_snpad"       
[10] "mez2"              "S_BedouinB-1"      "S_Burmese-1"      
[13] "S_Dai-2"           "S_Dinka-1"         "S_Finnish-2"      
[16] "S_French-1"        "S_Gambian-1"       "S_Han-2"          
[19] "S_Ju_hoan_North-1" "S_Karitiana-1"     "S_Mandenka-1"     
[22] "S_Mbuti-1"         "S_Papuan-2"        "S_Punjabi-1"      
[25] "S_Saami-2"         "S_Sardinian-1"     "S_Thai-1"         
[28] "S_Turkish-1"       "S_Yoruba-2"        "shotgun_mez2"     
[31] "shotgun_spy1"      "spy1_snpad"        "spy1"             
[34] "ustishim"

In [4]:
bed_full <- here("data/coord/capture_full.bed")

In [5]:
capture_full <- import.bed(bed_full)

In [6]:
sum(width(capture_full))

[1] 6912728

In [7]:
counts_full <- future_map_dfr(samples_full, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 0, maxdp = 1, bed_filter = bed_full)
    gt_nodmg <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 0, maxdp = 1, nodmg = T, bed_filter = bed_full)

    tibble(
        individual = ind,
        total = sum(width(capture_full)),
        all = sum(!is.na(gt_all[[ind]])),
        all_prop = all / total,
        nodmg = sum(!is.na(gt_nodmg[[ind]])),
        nodmg_prop = nodmg / total
    )
})

In [8]:
mindp_counts_full <- future_map_dfr(samples_full, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 1, bed_filter = bed_full)
    gt_nodmg <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 1, nodmg = T, bed_filter = bed_full)

    tibble(
        individual = ind,
        total = sum(width(capture_full)),
        all = sum(!is.na(gt_all[[ind]])),
        all_prop = all / total,
        nodmg = sum(!is.na(gt_nodmg[[ind]])),
        nodmg_prop = nodmg / total
    )
})

In [9]:
maxdp_mindp_counts_full <- future_map_dfr(samples_full, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, bed_filter = bed_full)
    gt_nodmg <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, nodmg = T, bed_filter = bed_full)

    tibble(
        individual = ind,
        total = sum(width(capture_full)),
        all = sum(!is.na(gt_all[[ind]])),
        all_prop = all / total,
        nodmg = sum(!is.na(gt_nodmg[[ind]])),
        nodmg_prop = nodmg / total
    )
})

In [10]:
counts_full_archaic_df <- counts_full %>% filter(str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|shotgun)"))

In [11]:
mindp_counts_full_archaic_df <- mindp_counts_full %>% filter(str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|shotgun)"))

In [12]:
maxdp_mindp_counts_full_archaic_df <- maxdp_mindp_counts_full %>% filter(str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|shotgun)"))

In [13]:
full_unfiltered <- counts_full_archaic_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.1f%%", 100 * .)) %>%
    select(-total) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F)) %>%
    mutate(filter = "unfiltered")

In [14]:
full_mindp <- mindp_counts_full_archaic_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.1f%%", 100 * .)) %>%
    select(-total) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F)) %>%
    mutate(filter = "min DP >= 3")

In [15]:
full_maxdp <- maxdp_mindp_counts_full_archaic_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.1f%%", 100 * .)) %>%
    select(-total) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F)) %>%
    mutate(filter = "min DP >= 3 and max DP <= 98% quantile")

In [16]:
bind_rows(full_unfiltered, full_mindp, full_maxdp)

name,all,all_prop,nodmg,nodmg_prop,filter
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Spy 94a (shotgun),"2,664,787",38.5%,"2,636,405",38.1%,unfiltered
Spy 94a,"3,502,380",50.7%,"3,470,671",50.2%,unfiltered
Spy 94a (snpAD),"3,556,151",51.4%,"3,522,222",51.0%,unfiltered
Mezmaiskaya 2 (shotgun),"3,822,106",55.3%,"3,762,768",54.4%,unfiltered
Denisova 4,"4,731,302",68.4%,"4,705,860",68.1%,unfiltered
Denisova 4 (snpAD),"4,823,855",69.8%,"4,797,798",69.4%,unfiltered
Denisova 8,"5,851,332",84.6%,"5,828,356",84.3%,unfiltered
Denisova 8 (snpAD),"6,259,573",90.6%,"6,235,802",90.2%,unfiltered
Mezmaiskaya 2,"6,348,948",91.8%,"6,346,684",91.8%,unfiltered


# 560 kb capture of regions from Lippold et al.

In [17]:
samples_lippold <- list.files(here("data/vcf"), "lippold_.*.vcf.gz$") %>%
    str_subset("mez2_dp|modern|merged|elsidron2_dp", negate = T) %>%
    str_replace("lippold_(.*).vcf.gz", "\\1")

In [18]:
samples_lippold <- "elsidron2"

In [19]:
bed_lippold <- here("data/coord/capture_lippold.bed")

In [20]:
capture_lippold <- import.bed(bed_lippold)

In [21]:
sum(width(capture_lippold))

[1] 556259

In [22]:
counts_lippold <- future_map_dfr(samples_lippold, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 0, maxdp = 1, bed_filter = bed_lippold)
    gt_nodmg <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 0, maxdp = 1, nodmg = T, bed_filter = bed_lippold)

    tibble(
        individual = ind,
        total = sum(width(capture_lippold)),
        all = sum(!is.na(gt_all[[ind]])),
        all_prop = all / total,
        nodmg = sum(!is.na(gt_nodmg[[ind]])),
        nodmg_prop = nodmg / total
    )
})

In [23]:
mindp_counts_lippold <- future_map_dfr(samples_lippold, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 3, maxdp = 1, bed_filter = bed_lippold)
    gt_nodmg <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 3, maxdp = 1, nodmg = T, bed_filter = bed_lippold)

    tibble(
        individual = ind,
        total = sum(width(capture_lippold)),
        all = sum(!is.na(gt_all[[ind]])),
        all_prop = all / total,
        nodmg = sum(!is.na(gt_nodmg[[ind]])),
        nodmg_prop = nodmg / total
    )
})

In [24]:
maxdp_mindp_counts_lippold <- future_map_dfr(samples_lippold, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, bed_filter = bed_lippold)
    gt_nodmg <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, nodmg = T, bed_filter = bed_lippold)

    tibble(
        individual = ind,
        total = sum(width(capture_lippold)),
        all = sum(!is.na(gt_all[[ind]])),
        all_prop = all / total,
        nodmg = sum(!is.na(gt_nodmg[[ind]])),
        nodmg_prop = nodmg / total
    )
})

In [25]:
counts_lippold_archaic_df <- counts_lippold %>% filter(str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|shotgun|elsidron)"))

In [26]:
mindp_counts_lippold_archaic_df <- mindp_counts_lippold %>% filter(str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|shotgun|elsidron)"))

In [27]:
maxdp_mindp_counts_lippold_archaic_df <- maxdp_mindp_counts_lippold %>% filter(str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|shotgun|elsidron)"))

In [28]:
lippold_unfiltered <- counts_lippold_archaic_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.1f%%", 100 * .)) %>%
    select(-total) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F)) %>%
    mutate(filter = "unfiltered")

In [29]:
lippold_mindp <- mindp_counts_lippold_archaic_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.1f%%", 100 * .)) %>%
    select(-total) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F)) %>%
    mutate(filter = "min DP >= 3")

In [30]:
lippold_maxdp <- maxdp_mindp_counts_lippold_archaic_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.1f%%", 100 * .)) %>%
    select(-total) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F)) %>%
    mutate(filter = "min DP >= 3 and max DP <= 98% quantile")

In [31]:
bind_rows(lippold_unfiltered, lippold_mindp, lippold_maxdp)

name,all,all_prop,nodmg,nodmg_prop,filter
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
El Sidrón 1253 (560 kb),"530,172",95.3%,"530,046",95.3%,unfiltered
El Sidrón 1253 (560 kb),"477,397",85.8%,"477,293",85.8%,min DP >= 3
El Sidrón 1253 (560 kb),"467,563",84.1%,"467,461",84.0%,min DP >= 3 and max DP <= 98% quantile
